In [4]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.utils import ImageReader
from PIL import Image
import io

class ImageTextPDFGenerator:
    def __init__(self, output_filename, row_height=100):  # Reduced row height
        self.output_filename = output_filename
        self.row_height = row_height
        self.page_width, self.page_height = A4
        self.margin = 30  # Reduced margin
        
        # Define action words (replace with your actual action words)
        self.action_words = ["UP", "DOWN", "LEFT", "RIGHT", "JUMP", "DUCK"]
        
        # Calculate dimensions
        self.usable_width = self.page_width - (2 * self.margin)
        self.image_width = self.usable_width / 6  # Images take up 1/6 of usable width
        self.image_height = row_height - 20  # Slightly smaller than row height
        
        # Initialize PDF
        self.c = canvas.Canvas(output_filename, pagesize=A4)
        self.current_y = self.page_height - self.margin
        
        # Add arrow symbol
        self.arrow_width = 30
        
    def _add_arrow(self, x, y):
        """Draw an arrow symbol"""
        self.c.setStrokeColor('black')
        self.c.setLineWidth(2)
        # Draw arrow line
        self.c.line(x, y, x + self.arrow_width, y)
        # Draw arrow head
        self.c.line(x + self.arrow_width - 10, y + 5, x + self.arrow_width, y)
        self.c.line(x + self.arrow_width - 10, y - 5, x + self.arrow_width, y)
        
    def _add_vertical_separator(self, x, y, height):
        """Draw a vertical separator line"""
        self.c.setStrokeColor('black')
        self.c.setLineWidth(1)
        self.c.line(x, y, x, y + height)

    def add_row(self, x_obs: Image.Image, x_action: int, y_obs: Image.Image, pred_obs: Image.Image):
        """
        Add a row with observation images and action text.
        
        Args:
            x_obs: Input observation (PIL Image)
            x_action: Action index
            y_obs: Target observation (PIL Image)
            pred_obs: Predicted observation (PIL Image)
        """
        # Check if we need a new page
        if self.current_y - self.row_height < self.margin:
            self.c.showPage()
            self.current_y = self.page_height - self.margin
        
        # Convert action index to word
        action_text = self.action_words[x_action]
        
        # Calculate positions
        x_positions = [
            self.margin,  # x_obs
            self.margin + self.image_width + 20,  # action text
            self.margin + (self.image_width + 20) * 2,  # arrow
            self.margin + (self.image_width + 20) * 3,  # y_obs
            self.margin + (self.image_width + 20) * 4,  # separator
            self.margin + (self.image_width + 20) * 4 + 20,  # pred_obs
        ]
        
        # Draw images and elements
        for i, (img, pos) in enumerate([(x_obs, 0), (y_obs, 3), (pred_obs, 5)]):
            img_buffer = io.BytesIO()
            img.save(img_buffer, format='PNG')
            img_buffer.seek(0)
            
            self.c.drawImage(
                ImageReader(img_buffer),
                x_positions[pos],
                self.current_y - self.image_height,
                width=self.image_width,
                height=self.image_height,
                preserveAspectRatio=True
            )
        
        # Add action text
        self.c.setFont("Helvetica", 12)
        text_width = self.c.stringWidth(action_text, "Helvetica", 12)
        text_x = x_positions[1] + (self.image_width - text_width) / 2
        text_y = self.current_y - self.image_height/2
        self.c.drawString(text_x, text_y, action_text)
        
        # Add arrow
        self._add_arrow(x_positions[2], self.current_y - self.image_height/2)
        
        # Add vertical separator
        self._add_vertical_separator(
            x_positions[4],
            self.current_y - self.image_height,
            self.image_height
        )
        
        # Update current_y position
        self.current_y -= self.row_height
        
    def save(self):
        """Save and close the PDF."""
        self.c.save()

# Example usage
def create_sample_pdf():
    # Create sample images (in real usage, you'd load your own images)
    sample_image = Image.new('RGB', (100, 100), color='red')
    
    # Create PDF
    pdf_gen = ImageTextPDFGenerator('output.pdf')
    
    # Add multiple rows
    for i in range(20):  # Create 5 rows
        pdf_gen.add_row(sample_image, i % len(pdf_gen.action_words), sample_image, sample_image)
    
    pdf_gen.save()

if __name__ == "__main__":
    create_sample_pdf()